# Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import time

import re
#from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing


# Converting Json data to DataFrame

In [2]:
data = pd.read_json('computers_train_xlarge.json',lines = True)
data.to_csv('train.csv',index = False)
df = pd.read_csv('train.csv')
df


,id_left,category_left,cluster_id_left,id_right,category_right,cluster_id_right,label,pair_id,brand_left,brand_right,description_left,description_right,keyValuePairs_left,keyValuePairs_right,price_left,price_right,specTableContent_left,specTableContent_right,title_left,title_right
0,2551242,Computers_and_Accessories,79457,16272671,Computers_and_Accessories,79457,1,2551242#16272671,"""Corsair""@en",NaN,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...",NaN,"{'Memory Type': 'DDR4 (PC4-21300)', 'Capacity'...","{'Categorie': 'Geheugen intern', 'Merk': 'Cors...",NaN,NaN,Memory Type DDR4 (PC4-21300) Capacity 64GB (4...,Categorie Geheugen intern Merk Corsair Produc...,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P..."
1,16757469,Computers_and_Accessories,63553,16476204,Computers_and_Accessories,63553,1,16757469#16476204,"""HP Enterprise""","""HP Enterprise""","""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","{'Category': 'Proliant HardDrive', 'Sub-Catego...","{'Category': 'Proliant HardDrive', 'Sub-Catego...",NaN,"""CAD"", ""$757.74 CAD""",Specifications: Category Proliant HardDrive S...,Specifications: Category Proliant HardDrive S...,"""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null"""
2,232007,Computers_and_Accessories,212917,16442945,Computers_and_Accessories,14319765,0,232007#16442945,"""SanDisk""","""HP Enterprise""","""SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...","""Description:Genuine HPE 1GB FBD PC2-5300(2x5...",NaN,"{'Category': 'Proliant Memory', 'Sub-Category'...","""USD"", ""7.98""","""USD"", ""$69.95""",NaN,Specifications: Category Proliant Memory Sub-...,"""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ..."
3,2066119,Computers_and_Accessories,667772,12411100,Computers_and_Accessories,667772,1,2066119#12411100,NaN,"""KINGSTON""","""DISCO DURO INTERNO SOLIDO HDD SSD""@es",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3..."
4,6656540,Computers_and_Accessories,505997,2639431,Computers_and_Accessories,154118,0,6656540#2639431,NaN,"""Corsair""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68456,6493497,Computers_and_Accessories,222681,16149764,Computers_and_Accessories,502396,0,6493497#16149764,"""HP Enterprise""","""HP Enterprise""","""Description:5 x 300GB 2.5-inch Serial Attache...","""Description:2 x 72GB 2.5-inch Serial Attache...","{'Category': 'Proliant HardDrive', 'Sub-Catego...","{'Category': 'Proliant HardDrive', 'Sub-Catego...",NaN,"""$151.40"", ""USD""",Specifications: Category Proliant HardDrive S...,Specifications: Category Proliant HardDrive S...,"""DG0300FARVV HP 300-GB 6G 10K 2.5 DP SAS"", ""N...","""Null"" ""512743-001 HP 72-GB 6G 15K 2.5 DP SAS"""
68457,17075265,Computers_and_Accessories,944407,17346839,Computers_and_Accessories,825203,0,17075265#17346839,"""Samsung""",NaN,NaN,"""Cost-effective SSD featuring TurboWrite and ...",NaN,NaN,NaN,NaN,NaN,NaN,"""Samsung - 840 EVO 250GB 2.5"" Solid State Dri...","""SSD 750 EVO 2.5” SATA III 120GB ""@en"
68458,16408794,Computers_and_Accessories,144052,3675781,Computers_and_Accessories,51801,0,16408794#3675781,"""INTEL""@fr","""Intel""@en",NaN,"""Quad Core Technology, 3.6GHz clock speed, 8MB...",NaN,"{'Processor Number': 'I7 7700', 'Number of Cor...",NaN,NaN,NaN,Processor Number I7 7700 Number of Cores 4 Nu...,"""Socket H4 1151 - Coffee Lake Core i7-8700K 6...","""7th Generation Intel® Core™ i7 7700 3.6GHz S..."
68459,13925964,Computers_and_Accessories,43816,15659664,Computers_and_Accessories,478188,0,13925964#15659664,NaN,NaN,"""Built to WD's highest standards of quality an...",NaN,NaN,NaN,"""$"", "

In [3]:
df.columns

Index(['id_left', 'category_left', 'cluster_id_left', 'id_right',
       'category_right', 'cluster_id_right', 'label', 'pair_id', 'brand_left',
       'brand_right', 'description_left', 'description_right',
       'keyValuePairs_left', 'keyValuePairs_right', 'price_left',
       'price_right', 'specTableContent_left', 'specTableContent_right',
       'title_left', 'title_right'],
      dtype='object')

# Data Cleaning : Removing Unwanted Columns

In [4]:
df.drop(df.columns[[2,5,6,8,9,12,13,14,15,16,17]],axis=1,inplace =True)
df

,id_left,category_left,id_right,category_right,pair_id,description_left,description_right,title_left,title_right
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,2551242#16272671,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...",NaN,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P..."
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,16757469#16476204,"""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null"""
2,232007,Computers_and_Accessories,16442945,Computers_and_Accessories,232007#16442945,"""SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...","""Description:Genuine HPE 1GB FBD PC2-5300(2x5...","""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ..."
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,2066119#12411100,"""DISCO DURO INTERNO SOLIDO HDD SSD""@es",NaN,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3..."
4,6656540,Computers_and_Accessories,2639431,Computers_and_Accessories,6656540#2639431,NaN,NaN,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4..."
...,...,...,...,...,...,...,...,...,...
68456,6493497,Computers_and_Accessories,16149764,Computers_and_Accessories,6493497#16149764,"""Description:5 x 300GB 2.5-inch Serial Attache...","""Description:2 x 72GB 2.5-inch Serial Attache...","""DG0300FARVV HP 300-GB 6G 10K 2.5 DP SAS"", ""N...","""Null"" ""512743-001 HP 72-GB 6G 15K 2.5 DP SAS"""
68457,17075265,Computers_and_Accessories,17346839,Computers_and_Accessories,17075265#17346839,NaN,"""Cost-effective SSD featuring TurboWrite and ...","""Samsung - 840 EVO 250GB 2.5"" Solid State Dri...","""SSD 750 EVO 2.5” SATA III 120GB ""@en"
68458,16408794,Computers_and_Accessories,3675781,Computers_and_Accessories,16408794#3675781,NaN,"""Quad Core Technology, 3.6GHz clock speed, 8MB...","""Socket H4 1151 - Coffee Lake Core i7-8700K 6...","""7th Generation Intel® Core™ i7 7700 3.6GHz S..."
68459,13925964,Computers_and_Accessories,15659664,Computers_and_Accessories,13925964#15659664,"""Built to WD's highest standards of quality an...",NaN,"""WD Blue WD5000AZLX - hard drive 500 GB SATA ...","""m rock ships""@en-US ""M-ROCK Ships New Camera..."


# Data Analysis 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68461 entries, 0 to 68460
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_left            68461 non-null  int64 
 1   category_left      68461 non-null  object
 2   id_right           68461 non-null  int64 
 3   category_right     68461 non-null  object
 4   pair_id            68461 non-null  object
 5   description_left   47460 non-null  object
 6   description_right  48360 non-null  object
 7   title_left         68461 non-null  object
 8   title_right        68461 non-null  object
dtypes: int64(2), object(7)
memory usage: 4.7+ MB


In [6]:
df['category_left'].value_counts()

Computers_and_Accessories     64958
Other_Electronics              1115
Office_Products                 844
Video_Games                     575
Camera_and_Photo                286
Musical_Instruments             269
Luggage_and_Travel_Gear         256
Tools_and_Home_Improvement      108
Cellphones_and_Accessories       50
Name: category_left, dtype: int64

In [7]:
df['category_right'].value_counts()


Computers_and_Accessories     66972
Other_Electronics               463
Video_Games                     405
Office_Products                 262
Luggage_and_Travel_Gear         122
Cellphones_and_Accessories      109
Camera_and_Photo                 74
Musical_Instruments              44
Tools_and_Home_Improvement       10
Name: category_right, dtype: int64

In [8]:
df['id_left'].value_counts()


14619165    43
8364399     42
5411511     41
8167053     40
4927241     40
            ..
6810432      1
16476204     1
10969778     1
5872903      1
4604957      1
Name: id_left, Length: 4287, dtype: int64

In [9]:
(df['id_left'] < 0).sum()

0

In [10]:
df['id_right'].value_counts()

7900893     116
12050629    107
7440444     106
6443065      97
13889537     97
           ... 
5199342       1
16567684      1
16123988      1
13661066      1
7867351       1
Name: id_right, Length: 4144, dtype: int64

In [11]:
df['description_left'].isnull().sum()

21001

# Filling the Null values of Description Columns by Title

In [12]:
df.description_left.fillna(df.title_left, inplace = True)
df['description_left']

0        "DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...
1        "Description:2 x 72GB 2.5-inch Serial Attached...
2         "SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...
3                  "DISCO DURO INTERNO SOLIDO HDD SSD"@es 
4         "Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...
                               ...                        
68456    "Description:5 x 300GB 2.5-inch Serial Attache...
68457     "Samsung - 840 EVO 250GB 2.5" Solid State Dri...
68458     "Socket H4 1151 - Coffee Lake Core i7-8700K 6...
68459    "Built to WD's highest standards of quality an...
68460    "61 cm 250 cd / m² 1920 x 1080 Pixeles 5 ms LE...
Name: description_left, Length: 68461, dtype: object

In [13]:
df['description_left'].isnull().sum()

0

In [14]:
df['description_right'].isnull().sum()

20101

In [15]:
df.description_right.fillna(df.title_left, inplace = True)
df['description_right']

0         "Corsair Vengeance LPX Black 64GB (4x16GB) DD...
1         "Description:10 x 72GB 2.5-inch Serial Attach...
2         "Description:Genuine HPE 1GB FBD PC2-5300(2x5...
3         "DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...
4         "Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...
                               ...                        
68456     "Description:2 x 72GB 2.5-inch Serial Attache...
68457     "Cost-effective SSD featuring TurboWrite and ...
68458    "Quad Core Technology, 3.6GHz clock speed, 8MB...
68459     "WD Blue WD5000AZLX - hard drive 500 GB SATA ...
68460    "Longitud diagonal: 24 "; Tamaño: 16:9; Tecnol...
Name: description_right, Length: 68461, dtype: object

In [16]:
df['description_right'].isnull().sum()

0

In [29]:
def remove_spaces(text):
    text=text.strip()
    text=text.split()
    return ' '.join(text)

contraction = {'cause':'because',
              'aint': 'am not',
              'aren\'t': 'are not'}

def mapping_replacer(x,dic):
    for words in dic.keys():
        if ' ' + words + ' ' in x:
            x=x.replace(' '+ words +' ' ,' '+dic[words]+' ' )
    return x



In [30]:
#Stemming, lemmetisation and tokenisation
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

nltk.LancasterStemmer
ls = LancasterStemmer()
lem = WordNetLemmatizer()
def lexicon_normalization(text):
    words = word_tokenize(text) 
    
    
    # 1- Stemming
    words_stem = [ls.stem(w) for w in words]
    
    # 2- Lemmatization
    words_lem = [lem.lemmatize(w) for w in words_stem]
    return words_lem



In [31]:

#Handling stopwords
from collections import Counter
def remove_stopword(text):
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])
    return text


In [37]:


#Removing links, brackets, numbers, punctuations etc.

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    text = re.sub('<.*?>+', ' ', text)
    text = re.sub('_', ' ', text)
    text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\'','', text)
    text = re.sub('and', '', text)
    
    
    return text


In [38]:

#Handling stopwords
from collections import Counter
def remove_stopword(text):
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])
    return text


In [39]:
#dc=df['description_right']
#import re
#dc = dc.map(lambda x: re.sub(r'\W+', ' ', str(x)))
#dc = dc.replace(r'\W+', ' ', regex=True)

#dc=dc.apply(lambda x: mapping_replacer(x, contraction))

#dc=dc.apply(lambda x:clean_text(x))

#dc=dc.apply(lambda x: remove_stopword(x))

#dc=dc.apply(lambda x: lexicon_normalization(x))

#dc

In [40]:
import re
df['description_right'] = df['description_right'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df['description_right'] = df['description_right'].replace(r'\W+', ' ', regex=True)

df['description_right']=df['description_right'].apply(lambda x:mapping_replacer(x, contraction))

df['description_right']=df['description_right'].apply(lambda x:clean_text(x))

df['description_right']=df['description_right'].apply(lambda x:remove_stopword(x))

#df['description_right']=df['description_right'].apply(lambda x: lexicon_normalization(x))

NameError: name 'stopwords' is not defined

In [ ]:
import re
df['description_left'] = df['description_left'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df['description_left'] = df['description_left'].replace(r'\W+', ' ', regex=True)

df['description_left']=df['description_left'].apply(lambda x: mapping_replacer(x, contraction))

df['description_left']=df['description_left'].apply(lambda x:clean_text(x))

#df['description_left']=df['description_left'].apply(lambda x: remove_stopword(x))

#df['description_left']=df['description_left'].apply(lambda x: lexicon_normalization(x))

In [ ]:
df['description_right']

In [ ]:
df['description_left']

# Creating Target column 

In [ ]:
def id_target(n):
    if(n<0):
        return 0
    else:
        return 1

In [ ]:
df['target'] = df['id_right'].apply(id_target)
df

# Counting matched and not matched Products

In [ ]:
df['target'].value_counts()

#0 for negative id_right values
#1 for positive id_right values

# Comparing category left and category right

In [ ]:
df['category_match'] = np.where(df['category_left'] == df['category_right'],1,0)
df.head()

#1-Match / 0-Unmatch


In [ ]:
df['category_match'].value_counts()


In [ ]:
df.info()

In [ ]:
df.drop(df.index[df['category_match'] == 0],inplace = True)

In [ ]:
df['category_match'].value_counts()

In [ ]:
newdf=df.copy()
newdf.head()

# Applying Maching Numbers function to match the products Features 

In [ ]:
def matching_numbers(description_right, description_left):

    description_right = set(re.findall(r'[0-9]+', description_right))
    description_left = set(re.findall(r'[0-9]+', description_left))    
    union = description_right.union(description_left)
    intersection = description_right.intersection(description_left)

    if len(description_right)==0 and len(description_left) == 0:
        return 1
    else:
        return (len(intersection)/ len(union))


# Implementing Levenshtein Text similarity 

In [ ]:

import jellyfish as jf
def engineer_features(df):
    
    df['description_left'] = df['description_left'].str.lower()
    df['description_right'] = df['description_right'].str.lower()

    df['levenshtein_distance'] = df.apply(
    lambda x: jf.levenshtein_distance(x['description_right'], 
                                      x['description_left']), axis=1)

    df['matching_numbers'] = df.apply(
    lambda x: matching_numbers(x['description_right'], 
                               x['description_left']), axis=1)

    df['matching_numbers_log'] = (df['matching_numbers']+1).apply(np.log)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(value=0, inplace=True)

    return df


# All Required Features 

In [ ]:
df.head()

In [ ]:
df = engineer_features(df)
df = df[['description_left','description_right','levenshtein_distance','matching_numbers']]
df

In [ ]:
df=df.sort_values(by=['levenshtein_distance'], ascending=[False])
df

# Examining the mean ,count and max values of columns

In [ ]:
df.describe()

In [ ]:
df['match'] = np.where(((df['levenshtein_distance']<70) & (df['matching_numbers']>0.4)),1,0)
df['match'].value_counts()

In [ ]:
df

# Applying Train test Split For Finding Model Accuracy

In [42]:

X = df[['levenshtein_distance',
       'matching_numbers']].values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
y_train

KeyError: "None of [Index(['levenshtein_distance', 'matching_numbers'], dtype='object')] are in the [columns]"

In [ ]:
data1 = pd.read_json('computers_gs.json',lines = True)
data1.to_csv('test.csv',index = False)
df_test = pd.read_csv('test.csv')
df_test['label'].value_counts()


In [ ]:
df_test.columns

In [ ]:
df_test.drop(df_test.columns[[2,5,8,9,12,13,14,15,16,17]],axis=1,inplace =True)
df_test


In [ ]:
df_test['description_right'].isnull().sum()

In [ ]:
df_test.description_right.fillna(df_test.title_left, inplace = True)
df_test['description_right']

In [ ]:
df_test['description_right'].isnull().sum()

In [ ]:
df_test['description_left'].isnull().sum()

In [ ]:
df_test.description_left.fillna(df_test.title_left, inplace = True)
df_test['description_left']

In [ ]:
df_test['description_left'].isnull().sum()

In [ ]:
df_test['category_match'] = np.where(df_test['category_left'] == df_test['category_right'],1,0)
df_test

In [ ]:
df_test['category_match'].value_counts()

In [ ]:
df_test.drop(df_test.index[df_test['category_match'] == 0],inplace = True)

In [ ]:
df_test['category_match'].value_counts()

In [ ]:
df_test['description_right'] = df_test['description_right'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df_test['description_right'] = df_test['description_right'].replace(r'\W+', ' ', regex=True)

df_test['description_right']=df_test['description_right'].apply(lambda x: mapping_replacer(x, contraction))

df_test['description_right']=df_test['description_right'].apply(lambda x:clean_text(x))

#df_test['description_right']=df_test['description_right'].apply(lambda x: remove_stopword(x))

#df['description_right']=df['description_right'].apply(lambda x: lexicon_normalization(x))

In [ ]:
df_test['description_left'] = df_test['description_left'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df_test['description_left'] = df_test['description_left'].replace(r'\W+', ' ', regex=True)

df_test['description_left']=df_test['description_left'].apply(lambda x: mapping_replacer(x, contraction))

df_test['description_left']=df_test['description_left'].apply(lambda x:clean_text(x))

#df_test['description_left']=df_test['description_left'].apply(lambda x: remove_stopword(x))

#df['description_right']=df['description_right'].apply(lambda x: lexicon_normalization(x))

In [ ]:
df_test = engineer_features(df_test)
df_test = df_test[['description_left','description_right','levenshtein_distance','matching_numbers','label']]
df_test

In [ ]:
X_train=df[['levenshtein_distance','matching_numbers']]
X_test=df_test[['levenshtein_distance','matching_numbers']]
y_train=df['match']
y_test=df_test['label']
y_train

# Creating Function For Finding Confusion Matrix

In [ ]:
def get_confusion_matrix_values(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    return(cm[0][0], cm[0][1], cm[1][0], cm[1][1])

# Model Building: Decision Tree and Random Forest 

In [ ]:
classifiers = {
    "DecisionTreeClassifier":DecisionTreeClassifier(),
    "RandomForestClassifier":RandomForestClassifier(n_estimators=100,max_depth=4),

}

df_results = pd.DataFrame(columns=['model', 'accuracy', 'precision',
                                   'recall','true_pos','false_pos',
                                   'true_neg','false_neg'])

for key in classifiers:

    classifier = classifiers[key]
    model = classifier.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    classification = classification_report(y_test, y_pred, zero_division=0)
    tp, fp, fn, tn = get_confusion_matrix_values(y_test, y_pred)

    row = {'model': key,
           'accuracy': accuracy,
           'precision': precision,
           'recall': recall,
           'f1': f1,
           'true_pos': tp,
           'false_pos': fp,
           'true_neg': tn,
           'false_neg': fn,
          }
    df_results = df_results.append(row, ignore_index=True)

df_results.head(10)

In [ ]:
classifier=RandomForestClassifier(n_estimators=100)
model = classifier.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

# Estimating Results in Binary

In [ ]:
results = pd.DataFrame(data={'predictions': y_pred, 'actual': y_test})
results['result'] = np.where(results['predictions']==results['actual'], 1, 0)
results


# Final Results with Match and Not Matched Classification

In [ ]:
results['predictions'].replace(0, 'Not match',inplace=True)
results['predictions'].replace(1, 'Match',inplace=True)

results['actual'].replace(0, 'Not Match',inplace=True)
results['actual'].replace(1, 'Match',inplace=True)

results['result'].replace(0, 'False',inplace=True)
results['result'].replace(1, 'True',inplace=True)
results